In [1]:
import torch
import torch.nn as nn

In [112]:
# from models.common import *
from models.common import Conv, Concat, MP, SPPCSPC, RepConv
from models.yolo import IDetect
from utils.general import make_divisible
from copy import deepcopy

# libraries from ofa
from ofa.imagenet_classification.elastic_nn.modules.dynamic_layers import DynamicConvLayer, DynamicConv2d, DynamicBatchNorm2d

# TODO

* ELANBlock 상위 클래스 만들어서 기본 메소드 상속하는 방식

In [113]:
# from yolov7 in common.py
class Conv(nn.Module):
    # Standard convolution
    def __init__(self, c1, c2, k=1, s=1, p=None, g=1, act=True):  # ch_in, ch_out, kernel, stride, padding, groups
        super(Conv, self).__init__()
        p=k//2
        self.conv = nn.Conv2d(c1, c2, k, s, p, groups=g, bias=False)
        self.bn = nn.BatchNorm2d(c2)
        self.act = nn.SiLU() if act is True else (act if isinstance(act, nn.Module) else nn.Identity())

    def forward(self, x):
        return self.act(self.bn(self.conv(x)))

    def fuseforward(self, x):
        return self.act(self.conv(x))


class Concat(nn.Module):
    def __init__(self, dimension=1):
        super(Concat, self).__init__()
        self.d = dimension

    def forward(self, x):
        return torch.cat(x, self.d)


class DyConv(nn.Module):
    # Dynamic Convolution for elastic channel size
    def __init__(self, c1, c2, k=1, s=1, act=True):  # ch_in, ch_out, kernel, stride
        super(DyConv, self).__init__()
        self.conv = DynamicConv2d(c1, c2, k, s) # auto same padding
        self.bn = DynamicBatchNorm2d(c2)
        self.act = nn.SiLU() if act is True else (act if isinstance(act, nn.Module) else nn.Identity())

    def forward(self, x):
        return self.act(self.bn(self.conv(x)))

    def fuseforward(self, x):
        return self.act(self.conv(x))

In [6]:
class ELAN(nn.Module):
    def __init__(self, c1, c2, k, depth):
        super(ELAN, self).__init__()
        assert c1 % 2 == 0

In [199]:
class ELANBlock(nn.Module):
    def __init__(self, layers, depth):
        super(ELANBlock, self).__init__()
        self.layers = layers
        self.act_idx = [idx for idx in range(depth * 2) if (idx % 2 == 1 or idx == 0)] # it is only for Bbone
        self.act_idx = [idx for idx in range(depth + 1)]
    
    def forward(self, x, d=None):
        outputs = []
        for i, m in enumerate(self.layers):
            if i == 0:
                outputs.append(m(x))
            else:
                x = m(x)
                outputs.append(x)
                
        if d is not None:
            return torch.cat([outputs[i] for i in self.act_idx[:d+1]], dim=1)
        return torch.cat([outputs[i] for i in self.act_idx], dim=1)

In [214]:
model

Sequential(
  (0): Conv(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): SiLU()
  )
  (1): Conv(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): SiLU()
  )
  (2): Conv(
    (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): SiLU()
  )
  (3): Conv(
    (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): SiLU()
  )
  (4): BBoneELAN(
    (cv1): Conv(
      (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): Bat

In [207]:
b_block = BBoneELAN(c1=128, c2=64, k=3, depth=4)
depth = 3
act_idx = b_block.act_idx[depth]
layers = deepcopy(b_block.layers[:act_idx+1])
block = ELANBlock(layers, depth)

In [215]:
b_block
block

ELANBlock(
  (layers): Sequential(
    (0): Conv(
      (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (1): Conv(
      (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (2): Conv(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (3): Conv(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (4): Conv(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), 

In [196]:
input = torch.randn(1, 128, 64, 64)
block(input).shape

torch.Size([1, 192, 64, 64])

In [172]:
# ELANBlock for Backbone
class BBoneELAN(ELAN):
    def __init__(self, c1, c2, k, depth):
        super(BBoneELAN, self).__init__(c1, c2, k, depth)
        assert c1 % 2 == 0 and depth < 5
        c_ = int(c1 / 2)
        self.c2 = c2
        
        self.depth = depth
        
        layers = []
        
        # depth 1
        self.cv1 = Conv(c1, c2, 1, 1)
        self.cv2 = Conv(c1, c2, 1, 1)
        # depth 2
        self.cv3 = Conv(c2, c2, k, 1)
        self.cv4 = Conv(c2, c2, k, 1)
        # depth 3
        self.cv5 = Conv(c2, c2, k, 1)
        self.cv6 = Conv(c2, c2, k, 1)
        # depth 4
        self.cv7 = Conv(c2, c2, k, 1)
        self.cv8 = Conv(c2, c2, k, 1)
        
        self.act_idx = [0, 1, 3, 5, 7][:depth+1] 
    
        layers.append(self.cv1)
        layers.append(self.cv2)
        layers.append(self.cv3)
        layers.append(self.cv4)
        layers.append(self.cv5)
        layers.append(self.cv6)
        layers.append(self.cv7)
        layers.append(self.cv8)
        self.layers = nn.Sequential(*layers)
    
    def set_depth(self, depth):
        self.depth = depth
        self.out_C = self.c2 * (depth + 1)
    
    def forward_once(self, x, d=None):
        outputs = []
        for i, m in enumerate(self.layers):
            if i == 0:
                outputs.append(m(x))
            else:
                x = m(x)
                outputs.append(x)
                
        if d is not None:
            return torch.cat([outputs[i] for i in self.act_idx[:d+1]], dim=1)
        return torch.cat([outputs[i] for i in self.act_idx], dim=1)
    
    def forward(self, x, d=None):
        outputs = []
        # depth 1
        x1 = self.cv1(x)
        outputs.append(x1)
        x2 = self.cv2(x)    
        outputs.append(x2)
        # depth 2
        x3 = self.cv3(x2)
        outputs.append(x3)
        x4 = self.cv4(x3)
        outputs.append(x4)
        # depth 3
        x5 = self.cv5(x4)
        outputs.append(x5)
        x6 = self.cv6(x5)
        outputs.append(x6)
        # depth 4
        x7 = self.cv7(x6)
        outputs.append(x7)
        x8 = self.cv8(x7)
        outputs.append(x8)
        
        if d is not None:
            return torch.cat([outputs[i] for i in self.act_idx[:d+1]], dim=1)
        return torch.cat([outputs[i] for i in self.act_idx], dim=1)

In [173]:
input = torch.randn(1, 128, 64, 64)
block = BBoneELAN(c1=128, c2=64, k=3, depth=4)

In [178]:
block.forward_once(input).shape, block.forward_once(input, d=3).shape, block.forward_once(input, d=2).shape, block.forward_once(input, d=1).shape

(torch.Size([1, 320, 64, 64]),
 torch.Size([1, 256, 64, 64]),
 torch.Size([1, 192, 64, 64]),
 torch.Size([1, 128, 64, 64]))

In [174]:
block(input).shape, block(input, d=3).shape, block(input, d=2).shape, block(input, d=1).shape

(torch.Size([1, 320, 64, 64]),
 torch.Size([1, 256, 64, 64]),
 torch.Size([1, 192, 64, 64]),
 torch.Size([1, 128, 64, 64]))

In [132]:
# c2 * (depth + 1)
dyconv = DyConv(64*(4+1), 256, 3, 1)

In [136]:
dyconv(block(input)).shape, dyconv(block(input, d=3)).shape, dyconv(block(input, d=2)).shape, dyconv(block(input, d=1)).shape

(torch.Size([1, 256, 64, 64]),
 torch.Size([1, 256, 64, 64]),
 torch.Size([1, 256, 64, 64]),
 torch.Size([1, 256, 64, 64]))

In [ ]:
# we can even search the out channel size
dyconv.conv.active_out_channel = 32
dyconv(input).shape

torch.Size([1, 32, 128, 128])

In [140]:
# ELANBlock for Head
# there are differences about cardinality(path) and channel size
class HeadELAN(ELAN):
    def __init__(self, c1, c2, k, depth):
        super(HeadELAN, self).__init__(c1, c2, k, depth)
        assert c1 % 2 == 0 and c2 % 2 == 0 and depth < 6
        c_ = int(c2 / 2)
        self.c2 = c2
        self.depth = depth
        
        # depth 1
        self.cv1 = Conv(c1, c2, 1, 1)
        self.cv2 = Conv(c1, c2, 1, 1)
        # depth 2
        self.cv3 = Conv(c2, c_, k, 1)
        # depth 3
        self.cv4 = Conv(c_, c_, k, 1)
        # depth 4
        self.cv5 = Conv(c_, c_, k, 1)
        # depth 5
        self.cv6 = Conv(c_, c_, k, 1)
        
        self.act_idx = [0, 1, 2, 3, 4, 5, 6][:depth+1] 
    
    def forward(self, x, d=None):
        outputs = []
        # depth 1
        x1 = self.cv1(x)
        outputs.append(x1)
        x2 = self.cv2(x)    
        outputs.append(x2)
        # depth 2
        x3 = self.cv3(x2)
        outputs.append(x3)
        # depth 3
        x4 = self.cv4(x3)
        outputs.append(x4)
        # depth 4
        x5 = self.cv5(x4)
        outputs.append(x5)
        # depth 5
        x6 = self.cv6(x5)
        outputs.append(x6)
        
        if d is not None:
            return torch.cat([outputs[i] for i in self.act_idx[:d+1]], dim=1)
        return torch.cat([outputs[i] for i in self.act_idx], dim=1)

In [141]:
input = torch.randn(1, 512, 64, 64)
block = HeadELAN(c1=512, c2=256, k=3, depth=5)

In [142]:
block(input).shape, block(input, d=4).shape, block(input, d=3).shape, block(input, d=2).shape, block(input, d=1).shape

(torch.Size([1, 1024, 64, 64]),
 torch.Size([1, 896, 64, 64]),
 torch.Size([1, 768, 64, 64]),
 torch.Size([1, 640, 64, 64]),
 torch.Size([1, 512, 64, 64]))

In [143]:
# c2 // 2 * (depth + 3)
dyconv = DyConv(256//2*(5+3), 256, 3, 1)

In [144]:
dyconv(block(input)).shape, dyconv(block(input, d=3)).shape, dyconv(block(input, d=2)).shape, dyconv(block(input, d=1)).shape

(torch.Size([1, 256, 64, 64]),
 torch.Size([1, 256, 64, 64]),
 torch.Size([1, 256, 64, 64]),
 torch.Size([1, 256, 64, 64]))

In [ ]:
# we can even search the out channel size
dyconv.conv.active_out_channel = 32
dyconv(input).shape

torch.Size([1, 32, 128, 128])

# YOLOSuperNet

* parse_supernet 함수 정의
: BBoneELAN과 HeadELAN에 대한 c2 계산 방식이 상이함.

In [145]:
def parse_supernet(d, ch):  # model_dict, input_channels(3)
    print('\n%3s%18s%3s%10s  %-40s%-30s' % ('', 'from', 'n', 'params', 'module', 'arguments'))
    anchors, nc, gd, gw = d['anchors'], d['nc'], d['depth_multiple'], d['width_multiple']
    na = (len(anchors[0]) // 2) if isinstance(anchors, list) else anchors  # number of anchors
    no = na * (nc + 5)  # number of outputs = anchors * (classes + 5)

    layers, save, c2 = [], [], ch[-1]  # layers, savelist, ch out
    for i, (f, n, m, args) in enumerate(d['backbone'] + d['head']):  # from, number, module, args
        m = eval(m) if isinstance(m, str) else m  # eval strings
        for j, a in enumerate(args):
            try:
                args[j] = eval(a) if isinstance(a, str) else a  # eval strings
            except:
                pass

        n = max(round(n * gd), 1) if n > 1 else n  # depth gain
        if m in [nn.Conv2d, Conv, RepConv, SPPCSPC, DyConv]:
            c1, c2 = ch[f], args[0]
            if c2 != no:  # if not output
                c2 = make_divisible(c2 * gw, 8)

            args = [c1, c2, *args[1:]]
            if m in [SPPCSPC]:
                args.insert(2, n)  # number of repeats
                n = 1
        elif m is BBoneELAN:
            c1, c2 = ch[f], int(args[0]*(args[-1]+1))
            args = [c1, *args]
        elif m is HeadELAN:
            c1, c2 = ch[f], int((args[0]*2) + (args[0]/2 * (args[-1]-1)))
            args = [c1, *args]
        elif m is nn.BatchNorm2d:
            args = [ch[f]]
        elif m is Concat:
            c2 = sum([ch[x] for x in f])
        # elif m is Chuncat:
        #     c2 = sum([ch[x] for x in f])
        # elif m is Shortcut:
        #     c2 = ch[f[0]]
        # elif m is Foldcut:
        #     c2 = ch[f] // 2
        elif m in [IDetect]:
            args.append([ch[x] for x in f])
            if isinstance(args[1], int):  # number of anchors
                args[1] = [list(range(args[1] * 2))] * len(f)
        # elif m is ReOrg:
        #     c2 = ch[f] * 4
        # elif m is Contract:
        #     c2 = ch[f] * args[0] ** 2
        # elif m is Expand:
        #     c2 = ch[f] // args[0] ** 2
        else:
            c2 = ch[f]
        print(m)
        m_ = nn.Sequential(*[m(*args) for _ in range(n)]) if n > 1 else m(*args)  # module
        t = str(m)[8:-2].replace('__main__.', '')  # module type
        np = sum([x.numel() for x in m_.parameters()])  # number params
        m_.i, m_.f, m_.type, m_.np = i, f, t, np  # attach index, 'from' index, type, number params
        print('%3s%18s%3s%10.0f  %-40s%-30s' % (i, f, n, np, t, args))  # print
        save.extend(x % i for x in ([f] if isinstance(f, int) else f) if x != -1)  # append to savelist
        layers.append(m_)
        if i == 0:
            ch = []
        ch.append(c2)
    return nn.Sequential(*layers), sorted(save)

In [146]:
import yaml
yaml_file = '../yaml/yolov7_elan_test.yml'
with open(yaml_file) as f:
    yaml = yaml.load(f, Loader=yaml.SafeLoader)

In [103]:
# Define model
ch, nc, anchors = 3, None, None

ch = yaml['ch'] = yaml.get('ch', ch)  # input channels
if nc and nc != yaml['nc']:
    print(f"Overriding model.yaml nc={yaml['nc']} with nc={nc}")
    yaml['nc'] = nc  # override yaml value
if anchors:
    print(f'Overriding model.yaml anchors with anchors={anchors}')
    yaml['anchors'] = round(anchors)  # override yaml value

In [104]:
model, save = parse_supernet(deepcopy(yaml), ch=[ch])


                 from  n    params  module                                  arguments                     
<class '__main__.Conv'>
  0                -1  1       928  Conv                                    [3, 32, 3, 1]                 
<class '__main__.Conv'>
  1                -1  1     18560  Conv                                    [32, 64, 3, 2]                
<class '__main__.Conv'>
  2                -1  1     36992  Conv                                    [64, 64, 3, 1]                
<class '__main__.Conv'>
  3                -1  1     73984  Conv                                    [64, 128, 3, 2]               
<class '__main__.BBoneELAN'>
  4                -1  1    238592  BBoneELAN                               [128, 64, 3, 3]               
<class '__main__.Conv'>
  5                -1  1     66048  Conv                                    [256, 256, 1, 1]              
<class 'models.common.MP'>
  6                -1  1         0  models.common.MP                      

In [152]:
import yaml
yaml_file = '../yaml/yolov7_dynamicsupernet.yml'
with open(yaml_file) as f:
    config = yaml.load(f, Loader=yaml.SafeLoader)

In [153]:
# Define model
ch, nc, anchors = 3, None, None

ch = config['ch'] = config.get('ch', ch)  # input channels
if nc and nc != config['nc']:
    print(f"Overriding model.yaml nc={config['nc']} with nc={nc}")
    config['nc'] = nc  # override yaml value
if anchors:
    print(f'Overriding model.yaml anchors with anchors={anchors}')
    config['anchors'] = round(anchors)  # override yaml value

In [154]:
model, save = parse_supernet(deepcopy(config), ch=[ch])


                 from  n    params  module                                  arguments                     
<class '__main__.Conv'>
  0                -1  1       928  Conv                                    [3, 32, 3, 1]                 
<class '__main__.Conv'>
  1                -1  1     18560  Conv                                    [32, 64, 3, 2]                
<class '__main__.Conv'>
  2                -1  1     36992  Conv                                    [64, 64, 3, 1]                
<class '__main__.Conv'>
  3                -1  1     73984  Conv                                    [64, 128, 3, 2]               
<class '__main__.BBoneELAN'>
  4                -1  1    238592  BBoneELAN                               [128, 64, 3, 3]               
<class '__main__.DyConv'>
  5                -1  1     66048  DyConv                                  [256, 256, 1, 1]              
<class 'models.common.MP'>
  6                -1  1         0  models.common.MP                    

In [105]:
# 1. Superclass인 ELAN 블록으로 관리하는 법
yaml['depth_list']
elan_idx = 0
for i, m in enumerate(model):
    if isinstance(m, ELAN):
        print(m.depth)
        depth = runtime_depth[elan_idx]
        # elan_idx += 1
        break
    
# 2. BBone과 Head를 나누어서 관리하는 법
# for m in model:
#     if isinstance(m, BBoneELAN):
#         print(m.depth)
#     if isinstance(m, HeadELAN):
#         print(m.depth)

3


In [108]:
m.c2, runtime_depth[elan_idx]

(64, 3)

In [106]:
m

BBoneELAN(
  (cv1): Conv(
    (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): SiLU()
  )
  (cv2): Conv(
    (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): SiLU()
  )
  (cv3): Conv(
    (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): SiLU()
  )
  (cv4): Conv(
    (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): SiLU()
  )
  (cv5): Conv(
    (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=

In [89]:
model[i+1]

Conv(
  (conv): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act): SiLU()
)

In [ ]:
def adjust_width_next_layer():
    

In [71]:
import yaml
yaml_file = '../yaml/yolov7_supernet.yml'
with open(yaml_file) as f:
    config = yaml.load(f, Loader=yaml.SafeLoader)
    
config['backbone']

[[-1, 1, 'Conv', [32, 3, 1]],
 [-1, 1, 'Conv', [64, 3, 2]],
 [-1, 1, 'Conv', [64, 3, 1]],
 [-1, 1, 'Conv', [128, 3, 2]],
 [-1, 1, 'BBoneELAN', [64, 3, 3]],
 [-1, 1, 'Conv', [256, 1, 1]],
 [-1, 1, 'MP', []],
 [-1, 1, 'Conv', [128, 1, 1]],
 [-3, 1, 'Conv', [128, 1, 1]],
 [-1, 1, 'Conv', [128, 3, 2]],
 [[-1, -3], 1, 'Concat', [1]],
 [-1, 1, 'BBoneELAN', [128, 3, 3]],
 [-1, 1, 'Conv', [512, 1, 1]],
 [-1, 1, 'MP', []],
 [-1, 1, 'Conv', [256, 1, 1]],
 [-3, 1, 'Conv', [256, 1, 1]],
 [-1, 1, 'Conv', [256, 3, 2]],
 [[-1, -3], 1, 'Concat', [1]],
 [-1, 1, 'BBoneELAN', [256, 3, 3]],
 [-1, 1, 'Conv', [1024, 1, 1]],
 [-1, 1, 'MP', []],
 [-1, 1, 'Conv', [512, 1, 1]],
 [-3, 1, 'Conv', [512, 1, 1]],
 [-1, 1, 'Conv', [512, 3, 2]],
 [[-1, -3], 1, 'Concat', [1]],
 [-1, 1, 'BBoneELAN', [256, 3, 3]],
 [-1, 1, 'Conv', [1024, 1, 1]]]

In [ ]:
def set_active_subnet(d=None, **kwargs):
        runtime_depth = d

In [ ]:
def get_active_net(preserve_weight=True):
    
    

In [72]:
def get_active_net_config(yaml, runtime_depth): # self
    idx = 0
    d = deepcopy(yaml)
    
    for i, (f, n, m, args) in enumerate(d['backbone'] + d['head']):
        if 'ELAN'in m:
            args[-1] = runtime_depth[idx]
            idx += 1
    
    del d['depth_list']
            
    return d
    

In [73]:
runtime_depth = [3,2,2,1,5,5,5,3]
d = get_active_net_config(config, runtime_depth)

In [75]:
for i, (f, n, m, args) in enumerate(d['backbone'] + d['head']):
    if 'ELAN'in m:
        print(i, m, args)

4 BBoneELAN [64, 3, 3]
11 BBoneELAN [128, 3, 2]
18 BBoneELAN [256, 3, 2]
25 BBoneELAN [256, 3, 1]
32 HeadELAN [256, 3, 5]
38 HeadELAN [128, 3, 5]
45 HeadELAN [256, 3, 5]
52 HeadELAN [512, 3, 3]


In [270]:
m = model[-1]  # Detect()
# s = 256  # 2x min stride
# m.stride = torch.tensor([s / x.shape[-2] for x in forward(torch.zeros(1, ch, s, s))])  # forward
# check_anchor_order(m)
# m.anchors /= m.stride.view(-1, 1, 1)
# self.stride = m.stride
# self._initialize_biases()  # only run once

In [170]:
from models.common import * 
from models.yolo import * 

def forward_once(model, x, runtime_depth, profile=False):
    y, dt = [], []  # outputs
    elan_idx = 0
    for m in model:
        if m.f != -1:  # if not from previous layer
            x = y[m.f] if isinstance(m.f, int) else [x if j == -1 else y[j] for j in m.f]  # from earlier layers

        if profile:
            c = isinstance(m, (Detect, IDetect, IAuxDetect, IBin))
            o = thop.profile(m, inputs=(x.copy() if c else x,), verbose=False)[0] / 1E9 * 2 if thop else 0  # FLOPS
            for _ in range(10):
                m(x.copy() if c else x)
            t = time_synchronized()
            for _ in range(10):
                m(x.copy() if c else x)
            dt.append((time_synchronized() - t) * 100)
            print('%10.1f%10.0f%10.1fms %-40s' % (o, m.np, dt[-1], m.type))

        if isinstance(m, ELAN): # 
            depth = runtime_depth[elan_idx]
            elan_idx += 1
            x = m(x, d=depth)
        else:
            x = m(x)  # run
        
        y.append(x if m.i in save else None)  # save output

    if profile:
        print('%.1fms total' % sum(dt))
    return x

In [171]:
input = torch.randn(1, 3, 256, 256)
runtime_depth = [3,2,2,1,5,5,5,3]

forward_once(model, input, runtime_depth)

[tensor([[[[[ 1.62231e-01, -1.89186e-01, -3.34552e-01,  ..., -2.17327e-01,  5.57717e-02, -2.49949e-01],
            [ 5.83315e-01, -2.06370e-02,  2.21807e-02,  ..., -7.35443e-01, -2.08533e-01, -1.82800e-01],
            [-3.55718e-03, -2.72500e-01, -2.94646e-01,  ..., -2.12679e-01,  7.00525e-02, -4.28083e-02],
            ...,
            [ 1.89134e-01, -5.20411e-01, -8.45757e-02,  ..., -7.42623e-01,  1.87639e-01, -3.75235e-01],
            [ 2.38706e-01, -2.64785e-01,  8.81961e-02,  ..., -2.00510e-01, -7.95626e-02, -1.93366e-01],
            [ 2.33426e-01, -5.03569e-02, -3.87518e-01,  ..., -2.86597e-01,  1.40115e-01, -2.19368e-01]],
 
           [[-1.78110e-02,  7.72588e-02,  6.29165e-02,  ..., -5.71224e-01,  1.91088e-01,  2.82930e-02],
            [ 4.85656e-01,  1.63607e-01, -2.14044e-01,  ..., -2.12386e-01, -4.93584e-01,  3.77903e-01],
            [ 3.58451e-01, -5.28260e-01, -6.48423e-02,  ...,  7.46303e-02, -2.37714e-01,  1.11812e-01],
            ...,
            [-4.89497e-01, 

In [156]:
x = torch.randn(1, 3, 256, 256)

y, dt = [], []  # outputs
for i, m in enumerate(model):
    if m.f != -1:  # if not from previous layer
        x = y[m.f] if isinstance(m.f, int) else [x if j == -1 else y[j] for j in m.f]  # from earlier layers
        
    x = m(x)  # run
    
    y.append(x if m.i in save else None)  # save output

In [159]:
len(x)

3

## ETC

In [51]:
# ELANBlock for Backbone
class ELANBlock(nn.Module):
    def __init__(self, c1, k, depth):
        super(ELANBlock, self).__init__()
        assert c1 % 2 == 0 and depth < 5
        c_ = int(c1 / 2)
        c2 = int(c1 / 2 * (depth+1))
        self.depth = depth
        # depth 1
        self.cv1 = Conv(c1, c_, 1, 1)
        self.cv2 = Conv(c1, c_, 1, 1)
        # depth 2
        self.cv3 = Conv(c_, c_, 3, 1)
        self.cv4 = Conv(c_, c_, 3, 1)
        # depth 3
        self.cv5 = Conv(c_, c_, 3, 1)
        self.cv6 = Conv(c_, c_, 3, 1)
        # depth 4
        self.cv7 = Conv(c_, c_, 3, 1)
        self.cv8 = Conv(c_, c_, 3, 1)
        
        self.cat = Concat(dimension=1)
        
        self.blocks, self.act_idx = self.set_blocks(c1, k, s, depth)
    
    def set_blocks(self, c1, k, s, depth):
        c_ = int(c1 / 2)
        layers = []
        # depth에 따라 from 이 결정됨.
        # depth = 3
        act_idx = [0, 1, 3, 5, 7][:depth] 
        f = [-1, -3, -5, -6]
        for i in range(depth):
            if i == 0:
                # depth 1
                m1, m2 = Conv(c1, c_, 1, 1), Conv(c1, c_, 1, 1)
                m1.i, m1.f = i, 0
                m2.i, m2.f = i+1, 0
                layers.append(m1)
                layers.append(m2)
                # layers.append(Conv(c1, c_, 1, 1))
                # layers.append(Conv(c1, c_, 1, 1))
            else:
                # another depth
                m3, m4 = Conv(c_, c_, k, 1), Conv(c_, c_, k, 1)
                m3.i, m3.f = 2*i, 2*i-1
                m4.i, m4.f = 2*i+1, 2*i
                layers.append(m3)
                layers.append(m4)
                # layers.append(Conv(c_, c_, k, 1))
                # layers.append(Conv(c_, c_, k, 1))
                m = nn.Sequential(
                    Conv(c_, c_, k, 1),
                    Conv(c_, c_, k, 1)
                )
                
        
        layers.append(Concat(1))
        return nn.Sequential(*layers), act_idx
    
    # def forward(self, x):
    #     x1 = self.cv1(x)
    #     x2 = self.cv2(x)    
    #     # depth 2
    #     x3 = self.cv3(x2)
    #     x4 = self.cv4(x3)
    #     # depth 3
    #     x5 = self.cv5(x4)
    #     x6 = self.cv6(x5)
        
    #     x = self.depth
    #     return torch.cat([x1, x2, x4, x6], dim=1)
    
    def forward(self, x):
        outputs = []
        x1 = self.cv1(x)
        outputs.append(x1)
        x2 = self.cv2(x)    
        outputs.append(x2)
        # depth 2
        x3 = self.cv3(x2)
        outputs.append(x3)
        x4 = self.cv4(x3)
        outputs.append(x4)
        # depth 3
        x5 = self.cv5(x4)
        outputs.append(x5)
        x6 = self.cv6(x5)
        outputs.append(x6)
        # depth 4
        x7 = self.cv7(x6)
        outputs.append(x7)
        x8 = self.cv8(x7)
        outputs.append(x8)
        
        return torch.cat([outputs[i] for i in self.act_idx], dim=1) # 0, 1, 3, 5 
        # return torch.cat([x1, x2, x4, x6], dim=1)
    
    def forward_blocks(self, x, depth=3):
        self.saved = []
        output = []
        
        for i, block in enumerate(self.blocks):
            if isinstance(block.f, int):
                output.append(block(x))
            
        return self.cat(output[self.act_idx])            

        
    
        

In [79]:
input = torch.randn(1, 128, 64, 64)
block = ELANBlock(c1=128, k=3, depth=4)

In [80]:
block(input).shape

torch.Size([1, 320, 64, 64])

5.0